**Random Forest Classifier**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install mediapipe


In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif


In [ ]:
# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
# Function to calculate the angle between two vectors
def calculate_angle(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    angle = np.arccos(dot_product / (magnitude1 * magnitude2))
    return np.degrees(angle)

# Function to extract features and labels
def extract_features_and_labels(dataset_dir):
    features = []
    labels = []

    for label_folder, label in [("good", 1), ("bad", 0)]:
        label_dir = os.path.join(dataset_dir, label_folder)
        for filename in os.listdir(label_dir):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                # Load image
                image_path = os.path.join(label_dir, filename)
                image = cv2.imread(image_path)
                if image is None:
                    continue

                # Process image using Mediapipe Pose
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                result = pose.process(image_rgb)

                if result.pose_landmarks:
                    landmarks = result.pose_landmarks.landmark

                    # Get landmark coordinates
                    left_shoulder = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
                    right_shoulder = (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
                    nose = (landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                            landmarks[mp_pose.PoseLandmark.NOSE.value].y)

                    # Convert to absolute pixel coordinates
                    h, w, _ = image.shape
                    left_shoulder = (int(left_shoulder[0] * w), int(left_shoulder[1] * h))
                    right_shoulder = (int(right_shoulder[0] * w), int(right_shoulder[1] * h))
                    nose = (int(nose[0] * w), int(nose[1] * h))

                    # Calculate vectors
                    green_line = np.array(right_shoulder) - np.array(left_shoulder)
                    red_line = np.array([1, 0])  # Horizontal reference vector
                    blue_line = np.array(nose) - np.array([(left_shoulder[0] + right_shoulder[0]) / 2,
                                                           (left_shoulder[1] + right_shoulder[1]) / 2])

                    # Calculate angles
                    angle_red_green = calculate_angle(red_line, green_line)
                    angle_blue_green = calculate_angle(blue_line, green_line)

                    # Append to features and labels
                    features.append([angle_red_green, angle_blue_green])
                    labels.append(label)

    return np.array(features), np.array(labels)


In [ ]:
# Path to your dataset
dataset_dir = "/content/drive/MyDrive/Research/3lineDataset"  # Replace with your dataset path


In [ ]:
dataset_dir

'/content/drive/MyDrive/Research/3lineDataset'

In [ ]:
# Extract features and labels
features, labels = extract_features_and_labels(dataset_dir)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.803030303030303
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.82      0.84        40
           1       0.74      0.77      0.75        26

    accuracy                           0.80        66
   macro avg       0.79      0.80      0.80        66
weighted avg       0.80      0.80      0.80        66



In [ ]:
# Save the trained model
# import joblib
# joblib.dump(model, "posture_classifier.pkl")
# print("Model saved as posture_classifier.pkl")